Импорт пакетов

In [34]:
import numpy as np
from scipy import sparse
from scipy.sparse import linalg as sl

Использование модели Изинга для оптимизации такого разреза в графе, который пересекал бы максимальное число ребер(хуево сказано, погуглите если не поняли)

1D случай,тоесть граф представляет собой цепочку

Функция действия z матрицы Паули на k кубит 

Матрица nxn действует на вектор состояние кубитов 

In [35]:
def sigmaz_k(k: int, n: int) -> (sparse.csr_matrix):
    left_part = sparse.eye(2 ** k)
    right_part = sparse.eye(2 ** (n - 1 - k))

    return sparse.kron(
        sparse.kron(
            left_part,
            sparse.csr_matrix(np.array([[1, 0,], [0, -1,],]))
        ),
        right_part
    )

Вычисление гамильтониана системы 

$H = J \sum_{i=0}^{N-1} \sigma_i^z \sigma_{i+1}^z - h \sum_{i=0}^{N} \sigma_i^z$

In [41]:
def ising(j: float, h: float, n: int) -> (sparse.csr_matrix):
    res = sparse.csr_matrix((2 ** n, 2 ** n), dtype=np.complex64)

    for i in range(n - 1):
        res += j * sigmaz_k(i, n) * sigmaz_k(i + 1, n)
        res -= h * sigmaz_k(i, n)

    res -= h * sigmaz_k(n - 1, n)

    return res

Собственное значение гамильтониана(вывод первого значения)

In [64]:
op = ising(1, 0, 10)
solution = sl.eigs(op, which="SR", k=1, return_eigenvectors=True)
print(f"Energy: {solution[0][0]}")

Energy: (-9+4.3526467380328195e-17j)


Поиск разделения на две группы в задачи оптимизиции среза в графе(случай h = 0, тоесть без ограничений)

In [60]:
def probs2bit_str(probs: np.array) -> (str):
    size = int(np.log2(probs.shape[0]))
    bit_s_num = np.where(probs == probs.max())[0][0]
    s = f"{bit_s_num:b}"
    s = "0" * (size - len(s)) + s
    return s

probs = solution[1] * solution[1].conj()
print(probs2bit_str(probs))

1010101010


Случай с h = 2 (ограничение на одну группу для разреза, в два раза)

In [65]:
def external_field(j: float, h: float, n: int) -> (None):
    op = ising(j, h, n)
    solution = sl.eigs(op, which="SR", k=1, return_eigenvectors=True)
    print(f"Energy: {solution[0][0]}")

    probs = solution[1] * solution[1].conj()
    print(probs2bit_str(probs))

external_field(1, 2, 10)

Energy: (-11.000000000000012-4.201612788373631e-18j)
0101001010
